In [1]:
#Importing required libraries
import pandas as pd
import numpy as np
import seaborn as sn
import matplotlib as mpl
import matplotlib.pyplot as plt
#import warnings
#warning.filterwarnings('ignore')

from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.metrics import classification_report
from collections import Counter

# This section focuses on Cleaning the RedLight Camera Dataframe

In [2]:
#Load Redlight camera data from database extracts
df_RL=pd.read_csv("../DB_Extracts/red_light_camera.csv")
df_RL.head(5)

,id,INTERSECTION_ID,LINEAR_NAME_FULL_1,LINEAR_NAME_FULL_2,ID1,X,Y,LONGITUDE,LATITUDE,OBJECTID,...,ADDITIONAL_INFO,POLICE_DIVISION_1,WARD_1,WARD_3,WARD_2,WARD_4,NAME,DISTRICT,RLC,MAIN
0,1843,13465959.0,Richmond St E,Parliament St,1,NaN,NaN,NaN,NaN,1,...,NaN,51,Toronto Centre(13),NaN,NaN,NaN,Richmond St E And Parliament St,Toronto and East York,6001,RICHMOND ST E
1,1844,13467993.0,Lake Shore Blvd W,York St,2,NaN,NaN,NaN,NaN,2,...,WB LAKE SHORE BLVD,52,Spadina-Fort York(10),NaN,NaN,NaN,Lake Shore Blvd W And York St,Toronto and East York,6002,LAKE SHORE BLVD W
2,1845,13444656.0,Steeles Ave W,Carpenter Rd,3,NaN,NaN,NaN,NaN,3,...,NaN,32,York Centre(6),NaN,NaN,NaN,Steeles Ave W And Carpenter Rd / Private Acces...,North York,6003,STEELES AVE W
3,1846,13444138.0,Steeles Ave W,Hilda Ave,4,NaN,NaN,NaN,NaN,4,...,NaN,32,Willowdale(18),NaN,NaN,NaN,Steeles Ave W And Hilda Ave,North York,6004,STEELES AVE W
4,1847,13451893.0,Albion Rd,Silverstone Dr,5,NaN,NaN,NaN,NaN,5,...,NaN,23,Etobicoke North(1),NaN,NaN,NaN,Albion Rd And Silverstone Dr,Etobicoke York,6005,ALBION RD


In [3]:
#show column names for RedLight dataframe
list(df_RL.columns)

['id',
 'INTERSECTION_ID',
 'LINEAR_NAME_FULL_1',
 'LINEAR_NAME_FULL_2',
 'ID1',
 'X',
 'Y',
 'LONGITUDE',
 'LATITUDE',
 'OBJECTID',
 'geometry',
 'MID_BLOCK',
 'POLICE_DIVISION_2',
 'POLICE_DIVISION_3',
 'ACTIVATION_DATE',
 'ACTIVATION_TIME',
 'SIDE2',
 'CLIENT_STREET_2',
 'CLIENT_STREET_1',
 'SIDE1',
 'MI_PRINX',
 'PRIVATE_ACCESS',
 'TCS',
 'ADDITIONAL_INFO',
 'POLICE_DIVISION_1',
 'WARD_1',
 'WARD_3',
 'WARD_2',
 'WARD_4',
 'NAME',
 'DISTRICT',
 'RLC',
 'MAIN']

In [4]:
# Remove unneeded columns from RedLight dataframe
df_RL_clean1=df_RL.drop(['id','INTERSECTION_ID','LINEAR_NAME_FULL_1', 'LINEAR_NAME_FULL_2','ID1','X','Y','LONGITUDE',
 'LATITUDE','OBJECTID','MID_BLOCK','POLICE_DIVISION_2','POLICE_DIVISION_3','SIDE2','SIDE1','MI_PRINX','PRIVATE_ACCESS',
 'TCS', 'ADDITIONAL_INFO', 'POLICE_DIVISION_1', 'WARD_1', 'WARD_3', 'WARD_2', 'WARD_4','MAIN'], axis = 1)
df_RL_clean1.head(5)

,geometry,ACTIVATION_DATE,ACTIVATION_TIME,CLIENT_STREET_2,CLIENT_STREET_1,NAME,DISTRICT,RLC
0,"{'type': 'Point', 'coordinates': (-79.36402299...",2007-11-09,5:00:00 AM,PARLIAMENT ST,RICHMOND ST E,Richmond St E And Parliament St,Toronto and East York,6001
1,"{'type': 'Point', 'coordinates': (-79.38086800...",2007-11-09,5:00:00 AM,YORK ST,LAKE SHORE BLVD W,Lake Shore Blvd W And York St,Toronto and East York,6002
2,"{'type': 'Point', 'coordinates': (-79.44758899...",2007-11-09,5:00:00 AM,CARPENTER RD,STEELES AVE W,Steeles Ave W And Carpenter Rd / Private Acces...,North York,6003
3,"{'type': 'Point', 'coordinates': (-79.42926700...",2007-11-09,5:00:00 AM,HILDA AVE,STEELES AVE W,Steeles Ave W And Hilda Ave,North York,6004
4,"{'type': 'Point', 'coordinates': (-79.60009399...",2007-11-09,5:00:00 AM,SILVERSTONE DR,ALBION RD,Albion Rd And Silverstone Dr,Etobicoke York,6005


In [5]:
df_RL_clean1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 214 entries, 0 to 213
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   geometry         214 non-null    object
 1   ACTIVATION_DATE  214 non-null    object
 2   ACTIVATION_TIME  214 non-null    object
 3   CLIENT_STREET_2  214 non-null    object
 4   CLIENT_STREET_1  214 non-null    object
 5   NAME             214 non-null    object
 6   DISTRICT         214 non-null    object
 7   RLC              214 non-null    int64 
dtypes: int64(1), object(7)
memory usage: 13.5+ KB


In [6]:
RL=df_RL_clean1.shape
print(RL)

(214, 8)


In [7]:
#parse out geo-coordinates
import re
df_RL_clean1["geometry2"] = df_RL_clean1["geometry"].str.extract(r"\((.*?)\)")

In [8]:
print(df_RL_clean1['geometry2'])

0      -79.3640229981536, 43.65455901107341
1       -79.3808680070637, 43.6414600021636
2        -79.447588998966, 43.7920130014137
3       -79.4292670008865, 43.7960070062936
4       -79.6000939933117, 43.7429520012624
                       ...                 
209     -79.5091200039297, 43.7229899950159
210    -79.2083430021465, 43.78797200428681
211     -79.3502909970486, 43.6932410046036
212    -79.16878799798131, 43.8042250062252
213     -79.3109519999502, 43.7854770033652
Name: geometry2, Length: 214, dtype: object


In [9]:
# split geometry2 into two columns: long & Lat

df_RL_clean1[['Longitude', 'Latitude']] = df_RL_clean1['geometry2'].str.split(',', 1, expand=True)

print(df_RL_clean1['Longitude'], df_RL_clean1['Latitude'])

0       -79.3640229981536
1       -79.3808680070637
2        -79.447588998966
3       -79.4292670008865
4       -79.6000939933117
              ...        
209     -79.5091200039297
210     -79.2083430021465
211     -79.3502909970486
212    -79.16878799798131
213     -79.3109519999502
Name: Longitude, Length: 214, dtype: object 0       43.65455901107341
1        43.6414600021636
2        43.7920130014137
3        43.7960070062936
4        43.7429520012624
              ...        
209      43.7229899950159
210     43.78797200428681
211      43.6932410046036
212      43.8042250062252
213      43.7854770033652
Name: Latitude, Length: 214, dtype: object


In [10]:
## Standardize Lat & Long Length to match between the files
df_RL_clean1['Longitude']=df_RL_clean1.Longitude.map(lambda l: l[:8])

In [11]:
df_RL_clean1['Latitude']=df_RL_clean1.Latitude.map(lambda l: l[:8].replace(" ",''))

In [12]:
df_RL=df_RL_clean1
df_RL.head(5)

,geometry,ACTIVATION_DATE,ACTIVATION_TIME,CLIENT_STREET_2,CLIENT_STREET_1,NAME,DISTRICT,RLC,geometry2,Longitude,Latitude
0,"{'type': 'Point', 'coordinates': (-79.36402299...",2007-11-09,5:00:00 AM,PARLIAMENT ST,RICHMOND ST E,Richmond St E And Parliament St,Toronto and East York,6001,"-79.3640229981536, 43.65455901107341",-79.3640,43.6545
1,"{'type': 'Point', 'coordinates': (-79.38086800...",2007-11-09,5:00:00 AM,YORK ST,LAKE SHORE BLVD W,Lake Shore Blvd W And York St,Toronto and East York,6002,"-79.3808680070637, 43.6414600021636",-79.3808,43.6414
2,"{'type': 'Point', 'coordinates': (-79.44758899...",2007-11-09,5:00:00 AM,CARPENTER RD,STEELES AVE W,Steeles Ave W And Carpenter Rd / Private Acces...,North York,6003,"-79.447588998966, 43.7920130014137",-79.4475,43.7920
3,"{'type': 'Point', 'coordinates': (-79.42926700...",2007-11-09,5:00:00 AM,HILDA AVE,STEELES AVE W,Steeles Ave W And Hilda Ave,North York,6004,"-79.4292670008865, 43.7960070062936",-79.4292,43.7960
4,"{'type': 'Point', 'coordinates': (-79.60009399...",2007-11-09,5:00:00 AM,SILVERSTONE DR,ALBION RD,Albion Rd And Silverstone Dr,Etobicoke York,6005,"-79.6000939933117, 43.7429520012624",-79.6000,43.7429


In [13]:
# Remove unneeded columns from RedLight dataframe
df_RL=df_RL.drop(['geometry'], axis = 1)

In [14]:
#re-order the columns for readability
df_RL = df_RL[["RLC", "ACTIVATION_DATE","ACTIVATION_TIME","CLIENT_STREET_2","CLIENT_STREET_1","NAME","DISTRICT","geometry2","Longitude","Latitude"]]
df_RL.head(5)

,RLC,ACTIVATION_DATE,ACTIVATION_TIME,CLIENT_STREET_2,CLIENT_STREET_1,NAME,DISTRICT,geometry2,Longitude,Latitude
0,6001,2007-11-09,5:00:00 AM,PARLIAMENT ST,RICHMOND ST E,Richmond St E And Parliament St,Toronto and East York,"-79.3640229981536, 43.65455901107341",-79.3640,43.6545
1,6002,2007-11-09,5:00:00 AM,YORK ST,LAKE SHORE BLVD W,Lake Shore Blvd W And York St,Toronto and East York,"-79.3808680070637, 43.6414600021636",-79.3808,43.6414
2,6003,2007-11-09,5:00:00 AM,CARPENTER RD,STEELES AVE W,Steeles Ave W And Carpenter Rd / Private Acces...,North York,"-79.447588998966, 43.7920130014137",-79.4475,43.7920
3,6004,2007-11-09,5:00:00 AM,HILDA AVE,STEELES AVE W,Steeles Ave W And Hilda Ave,North York,"-79.4292670008865, 43.7960070062936",-79.4292,43.7960
4,6005,2007-11-09,5:00:00 AM,SILVERSTONE DR,ALBION RD,Albion Rd And Silverstone Dr,Etobicoke York,"-79.6000939933117, 43.7429520012624",-79.6000,43.7429


## This section focuses on Cleaning the Seriously Injured Dataframe

In [15]:
# This setting lets you see all columns in the output
pd.set_option('display.max_columns',None)

In [16]:
df_KSI=pd.read_csv("../DB_Extracts/seriously_injured_data.csv")
df_KSI

C:\Users\bardhonia.harding\Anaconda3\envs\mlenv\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (23) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,ACCNUM,ObjectId,INVTYPE,INVAGE,INJURY_ID,FATAL_NO,INITDIR,VEHTYPE,MANOEUVER,DRIVACT,DRIVCOND,PEDTYPE,PEDACT,PEDCOND,CYCLISTYPE,CYCACT,CYCCOND,PEDESTRIAN,CYCLIST,AUTOMOBILE,MOTORCYCLE,TRUCK,TRSN_CITY_VEH,EMERG_VEH,PASSENGER,SPEEDING,AG_DRIV,REDLIGHT,ALCOHOL,DISABILITY,YEAR,DATE,TIME,HOUR,STREET1,STREET2,OFFSET,ROAD_CLASS_ID,DISTRICT,WARDNUM,DIVISION,LOCCOORD,ACCLOC,TRAFFCTL,VISIBILITY,LIGHT,RDSFCOND,ACCLASS,IMPACTYPE,POLICE_DIVISION,HOOD_ID,NEIGHBOURHOOD,geometry
0,25301,16392,Driver,75 to 79,1.0,NaN,North,"Automobile, Station Wagon",Going Ahead,Driving Properly,Normal,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020,2020-01-04,1850,18,EGLINTON AVE E,BAYVIEW AVE,9 m North of,1.0,North York,15.0,53.0,Intersection,Intersection Related,Traffic Signal,Rain,Dark,Wet,Non-Fatal Injury,Pedestrian Collisions,D53,56,Leaside-Bennington (56),"{'type': 'Point', 'coordinates': (-79.377116, ..."
1,25301,16396,Pedestrian,65 to 69,4.0,NaN,East,NaN,NaN,NaN,NaN,Pedestrian hit at mid-block,"Crossing, no Traffic Control",Medical or Physical Disability,NaN,NaN,NaN,Yes,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020,2020-01-04,1850,18,EGLINTON AVE E,BAYVIEW AVE,9 m North of,1.0,North York,15.0,53.0,Intersection,Intersection Related,Traffic Signal,Rain,Dark,Wet,Non-Fatal Injury,Pedestrian Collisions,D53,56,Leaside-Bennington (56),"{'type': 'Point', 'coordinates': (-79.377116, ..."
2,26294,16402,Pedestrian,65 to 69,2.0,1.0,East,NaN,NaN,NaN,NaN,Pedestrian hit at mid-block,"Crossing, no Traffic Control",Unknown,NaN,NaN,NaN,Yes,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020,2020-01-04,2214,22,256 JARVIS ST,NaN,GERRARD ST E,1.0,Toronto and East York,13.0,51.0,Mid-Block,Non Intersection,No Control,Clear,"Dark, artificial",Dry,Fatal,Pedestrian Collisions,D51,73,Moss Park (73),"{'type': 'Point', 'coordinates': (-79.37556, 4..."
3,26294,16407,Driver,20 to 24,1.0,NaN,North,"Automobile, Station Wagon",Going Ahead,Driving Properly,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020,2020-01-04,2214,22,256 JARVIS ST,NaN,GERRARD ST E,1.0,Toronto and East York,13.0,51.0,Mid-Block,Non Intersection,No Control,Clear,"Dark, artificial",Dry,Fatal,Pedestrian Collisions,D51,73,Moss Park (73),"{'type': 'Point', 'coordinates': (-79.37556, 4..."
4,37330,16506,Driver,50 to 54,1.0,NaN,West,"Automobile, Station Wagon",Going Ahead,Failed to Yield Right of Way,Normal,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,2020,2020-01-06,1555,15,DUNDAS ST W,SHERIDAN AVE,6 m South of,3.0,Toronto East York,9.0,11.0,Intersection,At Intersection,Traffic Signal,Clear,"Dawn, artificial",Dry,Non-Fatal Injury,Pedestrian Collisions,D11,84,Little Portugal (84),"{'type': 'Point', 'coordinates': (-79.43365, 4..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16855,9002501134,16387,Pedestrian,50 to 54,4.0,NaN,Unknown,NaN,NaN,NaN,NaN,Vehicle turns left while ped crosses without R...,Crossing without right of way,Inattentive,NaN,NaN,NaN,Yes,NaN,Yes,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,NaN,2019,2019-12-28,1740,17,MARTIN GROVE RD,DIXON RD,NaN,1.0,Etobicoke York,NaN,23.0,Intersection,At Intersection,Traffic Signal,Clear,Dark,Dry,Non-Fatal Injury,Pedestrian Collisions,D23,1,West Humber-Clairville (1),"{'type': 'Point', 'coordinates': (-79.570796, ..."
16856,9085345312,15298,Driver,30 to 34,4.0,NaN,East,"Automobile, Station Wagon",Going Ahead,Lost control,Medical or Physical Disability,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,Yes,NaN,Yes,NaN,NaN,Yes,2019,2019-05-10,2200,22,DANFORTH AVE,CHESTER AVE,22 m East of,1.0,Toronto and East York,14.0,NaN,Intersection,Intersection Related,No Control,Clear,"Dark, artificial",Dry,Non-Fatal Injury,Turning Movement,D55,68,North Riverdale (68),"{'type': 'Point', 'coordinates': (-79.351924, .

In [17]:
# Number of rows and columns in each dataset
KSI=df_KSI.shape
print(KSI)

(16860, 53)


In [18]:
df_KSI.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16860 entries, 0 to 16859
Data columns (total 53 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ACCNUM           16860 non-null  int64  
 1   ObjectId         16860 non-null  int64  
 2   INVTYPE          16848 non-null  object 
 3   INVAGE           16860 non-null  object 
 4   INJURY_ID        15248 non-null  float64
 5   FATAL_NO         713 non-null    float64
 6   INITDIR          11966 non-null  object 
 7   VEHTYPE          14047 non-null  object 
 8   MANOEUVER        9627 non-null   object 
 9   DRIVACT          8462 non-null   object 
 10  DRIVCOND         8464 non-null   object 
 11  PEDTYPE          2786 non-null   object 
 12  PEDACT           2779 non-null   object 
 13  PEDCOND          2835 non-null   object 
 14  CYCLISTYPE       700 non-null    object 
 15  CYCACT           707 non-null    object 
 16  CYCCOND          706 non-null    object 
 17  PEDESTRIAN  

In [19]:
# Remove all rows where injury is Null as these are only Vehicle or property owners

df_KSI_clean1=df_KSI.dropna(subset=['INJURY_ID'])

KSI=df_KSI_clean1.shape
print(KSI)

(15248, 53)


In [20]:
# remove non-driver rows types

#define list of values
values = ["Cyclist","Cyclist Passenger","Pedestrian - Not Hit","In-Line Skater","Motorcycle Passenger","Other", "Other Property Owner","Passenger","Pedestrian","Vehicle Owner","Wheelchair"]

#drop any rows not equal to values
df_KSI_clean1 = df_KSI_clean1[df_KSI_clean1.INVTYPE.isin(values) == False]

In [21]:
# count unique values in "INVTYPE" column

df_KSI_clean1.groupby(['INVTYPE']).count()

,ACCNUM,ObjectId,INVAGE,INJURY_ID,FATAL_NO,INITDIR,VEHTYPE,MANOEUVER,DRIVACT,DRIVCOND,PEDTYPE,PEDACT,PEDCOND,CYCLISTYPE,CYCACT,CYCCOND,PEDESTRIAN,CYCLIST,AUTOMOBILE,MOTORCYCLE,TRUCK,TRSN_CITY_VEH,EMERG_VEH,PASSENGER,SPEEDING,AG_DRIV,REDLIGHT,ALCOHOL,DISABILITY,YEAR,DATE,TIME,HOUR,STREET1,STREET2,OFFSET,ROAD_CLASS_ID,DISTRICT,WARDNUM,DIVISION,LOCCOORD,ACCLOC,TRAFFCTL,VISIBILITY,LIGHT,RDSFCOND,ACCLASS,IMPACTYPE,POLICE_DIVISION,HOOD_ID,NEIGHBOURHOOD,geometry
INVTYPE,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Driver,7606,7606,7606,7606,129,7557,7549,7603,7507,7506,3,0,0,0,0,0,2689,695,7317,463,191,414,12,2565,1066,4153,734,346,277,7606,7606,7606,7606,7606,6931,1337,7383,7541,6448,6704,7557,5277,7592,7598,7606,7593,7606,7603,7606,7606,7606,7606
Driver - Not Hit,13,13,13,13,0,2,9,2,2,2,0,0,0,0,0,0,5,2,9,4,1,3,0,3,2,4,0,0,0,13,13,13,13,13,11,2,11,13,12,12,13,9,13,13,13,13,13,13,13,13,13,13
Moped Driver,27,27,27,27,0,27,27,26,21,21,0,0,0,0,0,0,0,27,19,0,0,1,0,4,1,12,0,0,0,27,27,27,27,27,25,0,27,27,22,26,27,13,27,27,27,27,27,27,27,27,27,27
Motorcycle Driver,607,607,607,607,63,607,607,607,607,607,0,0,0,0,0,0,23,4,432,607,12,7,0,102,134,360,28,12,3,607,607,607,607,607,554,160,569,592,525,534,606,450,607,606,607,606,607,607,607,607,607,607
Truck Driver,316,316,316,316,3,316,316,316,314,314,0,0,0,0,0,0,123,45,126,12,312,6,0,67,35,150,18,4,5,316,316,316,316,316,289,56,303,313,268,278,316,229,316,315,316,315,316,316,316,316,316,316


In [22]:
# count unique values in "INVTYPE" column

df_KSI_clean1.groupby(['ACCLASS']).count()

,ACCNUM,ObjectId,INVTYPE,INVAGE,INJURY_ID,FATAL_NO,INITDIR,VEHTYPE,MANOEUVER,DRIVACT,DRIVCOND,PEDTYPE,PEDACT,PEDCOND,CYCLISTYPE,CYCACT,CYCCOND,PEDESTRIAN,CYCLIST,AUTOMOBILE,MOTORCYCLE,TRUCK,TRSN_CITY_VEH,EMERG_VEH,PASSENGER,SPEEDING,AG_DRIV,REDLIGHT,ALCOHOL,DISABILITY,YEAR,DATE,TIME,HOUR,STREET1,STREET2,OFFSET,ROAD_CLASS_ID,DISTRICT,WARDNUM,DIVISION,LOCCOORD,ACCLOC,TRAFFCTL,VISIBILITY,LIGHT,RDSFCOND,IMPACTYPE,POLICE_DIVISION,HOOD_ID,NEIGHBOURHOOD,geometry
ACCLASS,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Fatal,1086,1086,1084,1086,1086,195,1080,1072,1080,1076,1075,1,0,0,0,0,0,492,51,942,133,133,86,0,334,254,552,100,57,32,1086,1086,1086,1086,1086,991,117,1045,1085,913,968,1086,742,1086,1076,1086,1071,1083,1086,1086,1086,1086
Non-Fatal Injury,7486,7486,7483,7486,7486,0,7430,7437,7475,7373,7373,2,0,0,0,0,0,2351,722,6962,953,384,345,12,2407,982,4125,680,305,253,7486,7486,7486,7486,7486,6822,1436,7251,7404,6364,6588,7436,5237,7472,7486,7486,7486,7486,7486,7486,7486,7486
Property Damage Only,2,2,2,2,2,0,2,2,2,2,2,0,0,0,0,0,0,0,0,2,0,0,0,0,0,2,2,0,0,0,2,2,2,2,2,0,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2


In [23]:
# storing unique value in a variable
unique_value = df_KSI_clean1["ACCNUM"].nunique()
  
# printing value
print(unique_value)

5937


In [24]:
# Remove duplicate accident data - raw data has a row per person involved in the accident.

# sorting by ACCNUM
df_KSI_clean1.sort_values("ACCNUM", inplace = True)
 
# dropping ALL duplicate ACCNUM rows
df_KSI_clean2=df_KSI_clean1.drop_duplicates(subset ="ACCNUM",keep = 'first')

df_KSI_clean2

,ACCNUM,ObjectId,INVTYPE,INVAGE,INJURY_ID,FATAL_NO,INITDIR,VEHTYPE,MANOEUVER,DRIVACT,DRIVCOND,PEDTYPE,PEDACT,PEDCOND,CYCLISTYPE,CYCACT,CYCCOND,PEDESTRIAN,CYCLIST,AUTOMOBILE,MOTORCYCLE,TRUCK,TRSN_CITY_VEH,EMERG_VEH,PASSENGER,SPEEDING,AG_DRIV,REDLIGHT,ALCOHOL,DISABILITY,YEAR,DATE,TIME,HOUR,STREET1,STREET2,OFFSET,ROAD_CLASS_ID,DISTRICT,WARDNUM,DIVISION,LOCCOORD,ACCLOC,TRAFFCTL,VISIBILITY,LIGHT,RDSFCOND,ACCLASS,IMPACTYPE,POLICE_DIVISION,HOOD_ID,NEIGHBOURHOOD,geometry
0,25301,16392,Driver,75 to 79,1.0,NaN,North,"Automobile, Station Wagon",Going Ahead,Driving Properly,Normal,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020,2020-01-04,1850,18,EGLINTON AVE E,BAYVIEW AVE,9 m North of,1.0,North York,15.0,53.0,Intersection,Intersection Related,Traffic Signal,Rain,Dark,Wet,Non-Fatal Injury,Pedestrian Collisions,D53,56,Leaside-Bennington (56),"{'type': 'Point', 'coordinates': (-79.377116, ..."
3,26294,16407,Driver,20 to 24,1.0,NaN,North,"Automobile, Station Wagon",Going Ahead,Driving Properly,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020,2020-01-04,2214,22,256 JARVIS ST,NaN,GERRARD ST E,1.0,Toronto and East York,13.0,51.0,Mid-Block,Non Intersection,No Control,Clear,"Dark, artificial",Dry,Fatal,Pedestrian Collisions,D51,73,Moss Park (73),"{'type': 'Point', 'coordinates': (-79.37556, 4..."
4,37330,16506,Driver,50 to 54,1.0,NaN,West,"Automobile, Station Wagon",Going Ahead,Failed to Yield Right of Way,Normal,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,2020,2020-01-06,1555,15,DUNDAS ST W,SHERIDAN AVE,6 m South of,3.0,Toronto East York,9.0,11.0,Intersection,At Intersection,Traffic Signal,Clear,"Dawn, artificial",Dry,Non-Fatal Injury,Pedestrian Collisions,D11,84,Little Portugal (84),"{'type': 'Point', 'coordinates': (-79.43365, 4..."
6,45664,16513,Driver,50 to 54,5.0,NaN,East,"Automobile, Station Wagon",Going Ahead,Driving Properly,Normal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,Yes,NaN,Yes,NaN,NaN,NaN,2020,2020-01-07,1850,18,EGLINTON AVE E,PHARMACY AVE,NaN,1.0,Scarborough,NaN,41.0,Intersection,At Intersection,Traffic Signal,Clear,Dark,Dry,Non-Fatal Injury,Turning Movement,D41,120,Clairlea-Birchmount (120),"{'type': 'Point', 'coordinates': (-79.297481, ..."
9,56815,16515,Driver,45 to 49,1.0,NaN,South,Delivery Van,Going Ahead,Disobeyed Traffic Control,Inattentive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,NaN,Yes,Yes,Yes,NaN,NaN,2020,2020-01-09,1100,11,MARTIN GROVE RD,BELFIELD RD,NaN,1.0,Etobicoke York,1.0,23.0,Intersection,At Intersection,Traffic Signal,Clear,Daylight,Dry,Non-Fatal Injury,Angle,D23,1,West Humber-Clairville (1),"{'type': 'Point', 'coordinates': (-79.575588, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16847,9002496688,16127,Driver,20 to 24,1.0,NaN,North,"Automobile, Station Wagon",Going Ahead,Driving Properly,Normal,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,2019,2019-12-27,2324,23,2901 JANE ST,NaN,3 m East of,1.0,Etobicoke York,7.0,31.0,Mid-Block,Non Intersection,No Control,Clear,"Dark, artificial",Dry,Non-Fatal Injury,Pedestrian Collisions,D31,25,Glenfield-Jane Heights (25),"{'type': 'Point', 'coordinates': (-79.51665, 4..."
16848,9002500056,16141,Driver,40 to 44,1.0,NaN,West,"Automobile, Station Wagon",Reversing,Other,Normal,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019,2019-12-28,1439,14,1325 ST CLAIR AVE W,NaN,NaN,1.0,Etobicoke York,9.0,13.0,Mid-Block,Non Intersection,No Control,Clear,Daylight,Dry,Non-Fatal Injury,Pedestrian Collisions,D13,92,Corso Italia-Davenport (92),"{'type': 'Point', 'coordinates': (-79.448985, ..."
16851,9002500260,16264,Driver,25 to 29,1.0,NaN,West,"Automobile, Station Wagon",Going Ahead,Driving Properly,Normal,NaN,NaN,NaN,NaN,NaN,NaN,Yes

In [25]:
#show column names for df_KSI_clean2 dataframe
list(df_KSI_clean2.columns)

['ACCNUM',
 'ObjectId',
 'INVTYPE',
 'INVAGE',
 'INJURY_ID',
 'FATAL_NO',
 'INITDIR',
 'VEHTYPE',
 'MANOEUVER',
 'DRIVACT',
 'DRIVCOND',
 'PEDTYPE',
 'PEDACT',
 'PEDCOND',
 'CYCLISTYPE',
 'CYCACT',
 'CYCCOND',
 'PEDESTRIAN',
 'CYCLIST',
 'AUTOMOBILE',
 'MOTORCYCLE',
 'TRUCK',
 'TRSN_CITY_VEH',
 'EMERG_VEH',
 'PASSENGER',
 'SPEEDING',
 'AG_DRIV',
 'REDLIGHT',
 'ALCOHOL',
 'DISABILITY',
 'YEAR',
 'DATE',
 'TIME',
 'HOUR',
 'STREET1',
 'STREET2',
 'OFFSET',
 'ROAD_CLASS_ID',
 'DISTRICT',
 'WARDNUM',
 'DIVISION',
 'LOCCOORD',
 'ACCLOC',
 'TRAFFCTL',
 'VISIBILITY',
 'LIGHT',
 'RDSFCOND',
 'ACCLASS',
 'IMPACTYPE',
 'POLICE_DIVISION',
 'HOOD_ID',
 'NEIGHBOURHOOD',
 'geometry']

In [26]:
#changing null values to Other in Imactype column for single record

df_KSI_clean2.loc[df_KSI_clean2['ACCNUM']==9002389784,"IMPACTYPE"]="Other"
df_KSI_clean2.loc[df_KSI_clean2['ACCNUM']==9002389784]

C:\Users\bardhonia.harding\Anaconda3\envs\mlenv\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,ACCNUM,ObjectId,INVTYPE,INVAGE,INJURY_ID,FATAL_NO,INITDIR,VEHTYPE,MANOEUVER,DRIVACT,DRIVCOND,PEDTYPE,PEDACT,PEDCOND,CYCLISTYPE,CYCACT,CYCCOND,PEDESTRIAN,CYCLIST,AUTOMOBILE,MOTORCYCLE,TRUCK,TRSN_CITY_VEH,EMERG_VEH,PASSENGER,SPEEDING,AG_DRIV,REDLIGHT,ALCOHOL,DISABILITY,YEAR,DATE,TIME,HOUR,STREET1,STREET2,OFFSET,ROAD_CLASS_ID,DISTRICT,WARDNUM,DIVISION,LOCCOORD,ACCLOC,TRAFFCTL,VISIBILITY,LIGHT,RDSFCOND,ACCLASS,IMPACTYPE,POLICE_DIVISION,HOOD_ID,NEIGHBOURHOOD,geometry
16794,9002389784,16651,Driver,70 to 74,5.0,NaN,North,"Automobile, Station Wagon",Turning Left,Improper Turn,Normal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,NaN,2019,2019-12-11,1843,18,REXDALE BLVD,QUEENS PLATE DR,NaN,1.0,Etobicoke York,NaN,NaN,Intersection,At Intersection,Traffic Signal,Clear,"Dark, artificial",Dry,Fatal,Other,D23,1,West Humber-Clairville (1),"{'type': 'Point', 'coordinates': (-79.60345, 4..."


In [27]:
#changing null values to Other in Road Surface condition column for select records

values={7000795019,7001983024,8000303628,8000662467,8000874551,8000879182,8000973371,8008069034,9000524060,9002403050}

for i in values:
    df_KSI_clean2.loc[df_KSI_clean2['ACCNUM']== i,"RDSFCOND"]="Other"


C:\Users\bardhonia.harding\Anaconda3\envs\mlenv\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [28]:
#changing null values to Other in Visibility column for select records

values={7001983024,8000303628,8000662467,8000874551,8000879182,8000973371,8008069034,9000419013,9002403050}

for i in values:
    df_KSI_clean2.loc[df_KSI_clean2['ACCNUM']== i,"VISIBILITY"]="Other"

In [29]:
# drop unneeded columns

df_KSI_clean3=df_KSI_clean2.drop(['OFFSET','DISTRICT','WARDNUM','DIVISION','ACCLOC','FATAL_NO',
 'INITDIR','PEDTYPE','PEDACT','PEDCOND','CYCLISTYPE','CYCACT','CYCCOND','POLICE_DIVISION','HOOD_ID','NEIGHBOURHOOD'], axis = 1)
df_KSI_clean3.head(5)

,ACCNUM,ObjectId,INVTYPE,INVAGE,INJURY_ID,VEHTYPE,MANOEUVER,DRIVACT,DRIVCOND,PEDESTRIAN,CYCLIST,AUTOMOBILE,MOTORCYCLE,TRUCK,TRSN_CITY_VEH,EMERG_VEH,PASSENGER,SPEEDING,AG_DRIV,REDLIGHT,ALCOHOL,DISABILITY,YEAR,DATE,TIME,HOUR,STREET1,STREET2,ROAD_CLASS_ID,LOCCOORD,TRAFFCTL,VISIBILITY,LIGHT,RDSFCOND,ACCLASS,IMPACTYPE,geometry
0,25301,16392,Driver,75 to 79,1.0,"Automobile, Station Wagon",Going Ahead,Driving Properly,Normal,Yes,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020,2020-01-04,1850,18,EGLINTON AVE E,BAYVIEW AVE,1.0,Intersection,Traffic Signal,Rain,Dark,Wet,Non-Fatal Injury,Pedestrian Collisions,"{'type': 'Point', 'coordinates': (-79.377116, ..."
3,26294,16407,Driver,20 to 24,1.0,"Automobile, Station Wagon",Going Ahead,Driving Properly,Unknown,Yes,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020,2020-01-04,2214,22,256 JARVIS ST,NaN,1.0,Mid-Block,No Control,Clear,"Dark, artificial",Dry,Fatal,Pedestrian Collisions,"{'type': 'Point', 'coordinates': (-79.37556, 4..."
4,37330,16506,Driver,50 to 54,1.0,"Automobile, Station Wagon",Going Ahead,Failed to Yield Right of Way,Normal,Yes,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,2020,2020-01-06,1555,15,DUNDAS ST W,SHERIDAN AVE,3.0,Intersection,Traffic Signal,Clear,"Dawn, artificial",Dry,Non-Fatal Injury,Pedestrian Collisions,"{'type': 'Point', 'coordinates': (-79.43365, 4..."
6,45664,16513,Driver,50 to 54,5.0,"Automobile, Station Wagon",Going Ahead,Driving Properly,Normal,NaN,NaN,Yes,NaN,NaN,NaN,NaN,Yes,NaN,Yes,NaN,NaN,NaN,2020,2020-01-07,1850,18,EGLINTON AVE E,PHARMACY AVE,1.0,Intersection,Traffic Signal,Clear,Dark,Dry,Non-Fatal Injury,Turning Movement,"{'type': 'Point', 'coordinates': (-79.297481, ..."
9,56815,16515,Driver,45 to 49,1.0,Delivery Van,Going Ahead,Disobeyed Traffic Control,Inattentive,NaN,NaN,Yes,NaN,NaN,NaN,NaN,NaN,Yes,Yes,Yes,NaN,NaN,2020,2020-01-09,1100,11,MARTIN GROVE RD,BELFIELD RD,1.0,Intersection,Traffic Signal,Clear,Daylight,Dry,Non-Fatal Injury,Angle,"{'type': 'Point', 'coordinates': (-79.575588, ..."


In [30]:
df_KSI_clean3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5937 entries, 0 to 16856
Data columns (total 37 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   ACCNUM         5937 non-null   int64  
 1   ObjectId       5937 non-null   int64  
 2   INVTYPE        5933 non-null   object 
 3   INVAGE         5937 non-null   object 
 4   INJURY_ID      5937 non-null   float64
 5   VEHTYPE        5888 non-null   object 
 6   MANOEUVER      5928 non-null   object 
 7   DRIVACT        5853 non-null   object 
 8   DRIVCOND       5852 non-null   object 
 9   PEDESTRIAN     2641 non-null   object 
 10  CYCLIST        714 non-null    object 
 11  AUTOMOBILE     5315 non-null   object 
 12  MOTORCYCLE     598 non-null    object 
 13  TRUCK          318 non-null    object 
 14  TRSN_CITY_VEH  308 non-null    object 
 15  EMERG_VEH      8 non-null      object 
 16  PASSENGER      1484 non-null   object 
 17  SPEEDING       709 non-null    object 
 18  AG_DRIV

In [31]:
#Convert all columns with Yes & Null Values to 1=Yes 0=Null


df_KSI_clean3['PEDESTRIAN'] = df_KSI_clean3['PEDESTRIAN'].apply(lambda x:1 if x == 'Yes' else 0)
df_KSI_clean3['CYCLIST'] = df_KSI_clean3['CYCLIST'].apply(lambda x:1 if x == 'Yes' else 0)
df_KSI_clean3['AUTOMOBILE'] = df_KSI_clean3['AUTOMOBILE'].apply(lambda x:1 if x == 'Yes' else 0)
df_KSI_clean3['MOTORCYCLE'] = df_KSI_clean3['MOTORCYCLE'].apply(lambda x:1 if x == 'Yes' else 0)
df_KSI_clean3['TRUCK'] = df_KSI_clean3['TRUCK'].apply(lambda x:1 if x == 'Yes' else 0)
df_KSI_clean3['TRSN_CITY_VEH'] = df_KSI_clean3['TRSN_CITY_VEH'].apply(lambda x:1 if x == 'Yes' else 0)
df_KSI_clean3['EMERG_VEH'] = df_KSI_clean3['EMERG_VEH'].apply(lambda x:1 if x == 'Yes' else 0)
df_KSI_clean3['PASSENGER'] = df_KSI_clean3['PASSENGER'].apply(lambda x:1 if x == 'Yes' else 0)
df_KSI_clean3['SPEEDING'] = df_KSI_clean3['SPEEDING'].apply(lambda x:1 if x == 'Yes' else 0)
df_KSI_clean3['AG_DRIV'] = df_KSI_clean3['AG_DRIV'].apply(lambda x:1 if x == 'Yes' else 0)
df_KSI_clean3['REDLIGHT'] = df_KSI_clean3['REDLIGHT'].apply(lambda x:1 if x == 'Yes' else 0)
df_KSI_clean3['ALCOHOL'] = df_KSI_clean3['ALCOHOL'].apply(lambda x:1 if x == 'Yes' else 0)
df_KSI_clean3['DISABILITY'] = df_KSI_clean3['DISABILITY'].apply(lambda x:1 if x == 'Yes' else 0)

print(df_KSI_clean3[['PEDESTRIAN','CYCLIST','AUTOMOBILE','MOTORCYCLE','TRUCK','TRSN_CITY_VEH','EMERG_VEH','PASSENGER','SPEEDING','AG_DRIV','REDLIGHT','ALCOHOL','ALCOHOL']]) 

       PEDESTRIAN  CYCLIST  AUTOMOBILE  MOTORCYCLE  TRUCK  TRSN_CITY_VEH  \
0               1        0           1           0      0              0   
3               1        0           1           0      0              0   
4               1        0           1           0      0              0   
6               0        0           1           0      0              0   
9               0        0           1           0      0              0   
...           ...      ...         ...         ...    ...            ...   
16847           1        0           1           0      0              0   
16848           1        0           1           0      0              0   
16851           1        0           1           0      0              0   
16853           1        0           1           0      0              0   
16856           0        0           1           0      0              0   

       EMERG_VEH  PASSENGER  SPEEDING  AG_DRIV  REDLIGHT  ALCOHOL  ALCOHOL  
0         

In [32]:
df_KSI_clean3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5937 entries, 0 to 16856
Data columns (total 37 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   ACCNUM         5937 non-null   int64  
 1   ObjectId       5937 non-null   int64  
 2   INVTYPE        5933 non-null   object 
 3   INVAGE         5937 non-null   object 
 4   INJURY_ID      5937 non-null   float64
 5   VEHTYPE        5888 non-null   object 
 6   MANOEUVER      5928 non-null   object 
 7   DRIVACT        5853 non-null   object 
 8   DRIVCOND       5852 non-null   object 
 9   PEDESTRIAN     5937 non-null   int64  
 10  CYCLIST        5937 non-null   int64  
 11  AUTOMOBILE     5937 non-null   int64  
 12  MOTORCYCLE     5937 non-null   int64  
 13  TRUCK          5937 non-null   int64  
 14  TRSN_CITY_VEH  5937 non-null   int64  
 15  EMERG_VEH      5937 non-null   int64  
 16  PASSENGER      5937 non-null   int64  
 17  SPEEDING       5937 non-null   int64  
 18  AG_DRIV

In [33]:
#parse out geo-coordinates

df_KSI_clean3["geometry2"] = df_KSI_clean3["geometry"].str.extract(r"\((.*?)\)")


In [34]:
print(df_KSI_clean3['geometry2'])

0        -79.377116, 43.710967
3         -79.37556, 43.659568
4         -79.43365, 43.649699
6        -79.297481, 43.725577
9        -79.575588, 43.701876
                 ...          
16847     -79.51665, 43.753241
16848    -79.448985, 43.676577
16851    -79.384853, 43.658438
16853    -79.570796, 43.691519
16856    -79.351924, 43.677485
Name: geometry2, Length: 5937, dtype: object


In [35]:
# split geometry2 into two columns: long & Lat

df_KSI_clean3[['Longitude', 'Latitude']] = df_KSI_clean3['geometry2'].str.split(',', 1, expand=True)

print(df_KSI_clean3[['Longitude','Latitude']])

        Longitude    Latitude
0      -79.377116   43.710967
3       -79.37556   43.659568
4       -79.43365   43.649699
6      -79.297481   43.725577
9      -79.575588   43.701876
...           ...         ...
16847   -79.51665   43.753241
16848  -79.448985   43.676577
16851  -79.384853   43.658438
16853  -79.570796   43.691519
16856  -79.351924   43.677485

[5937 rows x 2 columns]


In [36]:
df_KSI_clean3

,ACCNUM,ObjectId,INVTYPE,INVAGE,INJURY_ID,VEHTYPE,MANOEUVER,DRIVACT,DRIVCOND,PEDESTRIAN,CYCLIST,AUTOMOBILE,MOTORCYCLE,TRUCK,TRSN_CITY_VEH,EMERG_VEH,PASSENGER,SPEEDING,AG_DRIV,REDLIGHT,ALCOHOL,DISABILITY,YEAR,DATE,TIME,HOUR,STREET1,STREET2,ROAD_CLASS_ID,LOCCOORD,TRAFFCTL,VISIBILITY,LIGHT,RDSFCOND,ACCLASS,IMPACTYPE,geometry,geometry2,Longitude,Latitude
0,25301,16392,Driver,75 to 79,1.0,"Automobile, Station Wagon",Going Ahead,Driving Properly,Normal,1,0,1,0,0,0,0,0,0,0,0,0,0,2020,2020-01-04,1850,18,EGLINTON AVE E,BAYVIEW AVE,1.0,Intersection,Traffic Signal,Rain,Dark,Wet,Non-Fatal Injury,Pedestrian Collisions,"{'type': 'Point', 'coordinates': (-79.377116, ...","-79.377116, 43.710967",-79.377116,43.710967
3,26294,16407,Driver,20 to 24,1.0,"Automobile, Station Wagon",Going Ahead,Driving Properly,Unknown,1,0,1,0,0,0,0,0,0,0,0,0,0,2020,2020-01-04,2214,22,256 JARVIS ST,NaN,1.0,Mid-Block,No Control,Clear,"Dark, artificial",Dry,Fatal,Pedestrian Collisions,"{'type': 'Point', 'coordinates': (-79.37556, 4...","-79.37556, 43.659568",-79.37556,43.659568
4,37330,16506,Driver,50 to 54,1.0,"Automobile, Station Wagon",Going Ahead,Failed to Yield Right of Way,Normal,1,0,1,0,0,0,0,0,0,1,0,0,0,2020,2020-01-06,1555,15,DUNDAS ST W,SHERIDAN AVE,3.0,Intersection,Traffic Signal,Clear,"Dawn, artificial",Dry,Non-Fatal Injury,Pedestrian Collisions,"{'type': 'Point', 'coordinates': (-79.43365, 4...","-79.43365, 43.649699",-79.43365,43.649699
6,45664,16513,Driver,50 to 54,5.0,"Automobile, Station Wagon",Going Ahead,Driving Properly,Normal,0,0,1,0,0,0,0,1,0,1,0,0,0,2020,2020-01-07,1850,18,EGLINTON AVE E,PHARMACY AVE,1.0,Intersection,Traffic Signal,Clear,Dark,Dry,Non-Fatal Injury,Turning Movement,"{'type': 'Point', 'coordinates': (-79.297481, ...","-79.297481, 43.725577",-79.297481,43.725577
9,56815,16515,Driver,45 to 49,1.0,Delivery Van,Going Ahead,Disobeyed Traffic Control,Inattentive,0,0,1,0,0,0,0,0,1,1,1,0,0,2020,2020-01-09,1100,11,MARTIN GROVE RD,BELFIELD RD,1.0,Intersection,Traffic Signal,Clear,Daylight,Dry,Non-Fatal Injury,Angle,"{'type': 'Point', 'coordinates': (-79.575588, ...","-79.575588, 43.701876",-79.575588,43.701876
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16847,9002496688,16127,Driver,20 to 24,1.0,"Automobile, Station Wagon",Going Ahead,Driving Properly,Normal,1,0,1,0,0,0,0,0,0,1,0,0,0,2019,2019-12-27,2324,23,2901 JANE ST,NaN,1.0,Mid-Block,No Control,Clear,"Dark, artificial",Dry,Non-Fatal Injury,Pedestrian Collisions,"{'type': 'Point', 'coordinates': (-79.51665, 4...","-79.51665, 43.753241",-79.51665,43.753241
16848,9002500056,16141,Driver,40 to 44,1.0,"Automobile, Station Wagon",Reversing,Other,Normal,1,0,1,0,0,0,0,0,0,0,0,0,0,2019,2019-12-28,1439,14,1325 ST CLAIR AVE W,NaN,1.0,Mid-Block,No Control,Clear,Daylight,Dry,Non-Fatal Injury,Pedestrian Collisions,"{'type': 'Point', 'coordinates': (-79.448985, ...","-79.448985, 43.676577",-79.448985,43.676577
16851,9002500260,16264,Driver,25 to 29,1.0,"Automobile, Station Wagon",Going Ahead,Driving Properly,Normal,1,0,1,0,0,0,0,0,0,1,1,0,0,2019,2019-12-28,1514,15,GERRARD ST W,BAY ST,1.0,Intersection,Traffic Signal,Clear,Daylight,Dry,Non-Fatal Injury,Pedestrian Collisions,"{'type': 'Point', 'coordinates': (-79.384853, ...","-79.384853, 43.658438",-79.384853,43.658438
16853,9002501134,16279,Driver,20 to 24,1.0,"Automobile, Station Wagon",Turning Left,Driving Properly,Normal,1,0,1,0,0,0,0,1,0,0,0,0,0,2019,2019-12-28,1740,17,MARTIN GROVE RD,DIXON RD,1.0,Intersection,Traffic Signal,Clear,Dark,Dry,Non-Fatal Injury,Pedestrian Collisions,"{'type': 'Point', 'coordinates': (-79.570796, ...","-79.570796, 43.691519",-79.570796,43.691519


In [37]:
# Standardize Lat & Long Length to match between the files

df_KSI_clean3['Longitude']=df_KSI_clean3.Longitude.map(lambda l: l[:8])

df_KSI_clean3['Latitude']=df_KSI_clean3.Latitude.map(lambda l: l[:8].replace(" ",''))


In [38]:
df_KSI_clean3

,ACCNUM,ObjectId,INVTYPE,INVAGE,INJURY_ID,VEHTYPE,MANOEUVER,DRIVACT,DRIVCOND,PEDESTRIAN,CYCLIST,AUTOMOBILE,MOTORCYCLE,TRUCK,TRSN_CITY_VEH,EMERG_VEH,PASSENGER,SPEEDING,AG_DRIV,REDLIGHT,ALCOHOL,DISABILITY,YEAR,DATE,TIME,HOUR,STREET1,STREET2,ROAD_CLASS_ID,LOCCOORD,TRAFFCTL,VISIBILITY,LIGHT,RDSFCOND,ACCLASS,IMPACTYPE,geometry,geometry2,Longitude,Latitude
0,25301,16392,Driver,75 to 79,1.0,"Automobile, Station Wagon",Going Ahead,Driving Properly,Normal,1,0,1,0,0,0,0,0,0,0,0,0,0,2020,2020-01-04,1850,18,EGLINTON AVE E,BAYVIEW AVE,1.0,Intersection,Traffic Signal,Rain,Dark,Wet,Non-Fatal Injury,Pedestrian Collisions,"{'type': 'Point', 'coordinates': (-79.377116, ...","-79.377116, 43.710967",-79.3771,43.7109
3,26294,16407,Driver,20 to 24,1.0,"Automobile, Station Wagon",Going Ahead,Driving Properly,Unknown,1,0,1,0,0,0,0,0,0,0,0,0,0,2020,2020-01-04,2214,22,256 JARVIS ST,NaN,1.0,Mid-Block,No Control,Clear,"Dark, artificial",Dry,Fatal,Pedestrian Collisions,"{'type': 'Point', 'coordinates': (-79.37556, 4...","-79.37556, 43.659568",-79.3755,43.6595
4,37330,16506,Driver,50 to 54,1.0,"Automobile, Station Wagon",Going Ahead,Failed to Yield Right of Way,Normal,1,0,1,0,0,0,0,0,0,1,0,0,0,2020,2020-01-06,1555,15,DUNDAS ST W,SHERIDAN AVE,3.0,Intersection,Traffic Signal,Clear,"Dawn, artificial",Dry,Non-Fatal Injury,Pedestrian Collisions,"{'type': 'Point', 'coordinates': (-79.43365, 4...","-79.43365, 43.649699",-79.4336,43.6496
6,45664,16513,Driver,50 to 54,5.0,"Automobile, Station Wagon",Going Ahead,Driving Properly,Normal,0,0,1,0,0,0,0,1,0,1,0,0,0,2020,2020-01-07,1850,18,EGLINTON AVE E,PHARMACY AVE,1.0,Intersection,Traffic Signal,Clear,Dark,Dry,Non-Fatal Injury,Turning Movement,"{'type': 'Point', 'coordinates': (-79.297481, ...","-79.297481, 43.725577",-79.2974,43.7255
9,56815,16515,Driver,45 to 49,1.0,Delivery Van,Going Ahead,Disobeyed Traffic Control,Inattentive,0,0,1,0,0,0,0,0,1,1,1,0,0,2020,2020-01-09,1100,11,MARTIN GROVE RD,BELFIELD RD,1.0,Intersection,Traffic Signal,Clear,Daylight,Dry,Non-Fatal Injury,Angle,"{'type': 'Point', 'coordinates': (-79.575588, ...","-79.575588, 43.701876",-79.5755,43.7018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16847,9002496688,16127,Driver,20 to 24,1.0,"Automobile, Station Wagon",Going Ahead,Driving Properly,Normal,1,0,1,0,0,0,0,0,0,1,0,0,0,2019,2019-12-27,2324,23,2901 JANE ST,NaN,1.0,Mid-Block,No Control,Clear,"Dark, artificial",Dry,Non-Fatal Injury,Pedestrian Collisions,"{'type': 'Point', 'coordinates': (-79.51665, 4...","-79.51665, 43.753241",-79.5166,43.7532
16848,9002500056,16141,Driver,40 to 44,1.0,"Automobile, Station Wagon",Reversing,Other,Normal,1,0,1,0,0,0,0,0,0,0,0,0,0,2019,2019-12-28,1439,14,1325 ST CLAIR AVE W,NaN,1.0,Mid-Block,No Control,Clear,Daylight,Dry,Non-Fatal Injury,Pedestrian Collisions,"{'type': 'Point', 'coordinates': (-79.448985, ...","-79.448985, 43.676577",-79.4489,43.6765
16851,9002500260,16264,Driver,25 to 29,1.0,"Automobile, Station Wagon",Going Ahead,Driving Properly,Normal,1,0,1,0,0,0,0,0,0,1,1,0,0,2019,2019-12-28,1514,15,GERRARD ST W,BAY ST,1.0,Intersection,Traffic Signal,Clear,Daylight,Dry,Non-Fatal Injury,Pedestrian Collisions,"{'type': 'Point', 'coordinates': (-79.384853, ...","-79.384853, 43.658438",-79.3848,43.6584
16853,9002501134,16279,Driver,20 to 24,1.0,"Automobile, Station Wagon",Turning Left,Driving Properly,Normal,1,0,1,0,0,0,0,1,0,0,0,0,0,2019,2019-12-28,1740,17,MARTIN GROVE RD,DIXON RD,1.0,Intersection,Traffic Signal,Clear,Dark,Dry,Non-Fatal Injury,Pedestrian Collisions,"{'type': 'Point', 'coordinates': (-79.570796, ...","-79.570796, 43.691519",-79.5707,43.6915


In [39]:
# Remove null values in road_class_ID before changing column to int type
df_KSI_clean3.dropna(subset = ['ROAD_CLASS_ID'], inplace=True)

# converting 'Injury_ID' and 'ROAD_CLASS_ID' from float to int
df_KSI_clean3['INJURY_ID'] = df_KSI_clean3['INJURY_ID'].astype(int)
df_KSI_clean3['ROAD_CLASS_ID'] = df_KSI_clean3['ROAD_CLASS_ID'].astype(int)  

# displaying the datatypes
#display(df_KSI_clean3.dtypes)

# displaying the info
df_KSI_clean3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5776 entries, 0 to 16856
Data columns (total 40 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   ACCNUM         5776 non-null   int64 
 1   ObjectId       5776 non-null   int64 
 2   INVTYPE        5772 non-null   object
 3   INVAGE         5776 non-null   object
 4   INJURY_ID      5776 non-null   int32 
 5   VEHTYPE        5729 non-null   object
 6   MANOEUVER      5767 non-null   object
 7   DRIVACT        5695 non-null   object
 8   DRIVCOND       5693 non-null   object
 9   PEDESTRIAN     5776 non-null   int64 
 10  CYCLIST        5776 non-null   int64 
 11  AUTOMOBILE     5776 non-null   int64 
 12  MOTORCYCLE     5776 non-null   int64 
 13  TRUCK          5776 non-null   int64 
 14  TRSN_CITY_VEH  5776 non-null   int64 
 15  EMERG_VEH      5776 non-null   int64 
 16  PASSENGER      5776 non-null   int64 
 17  SPEEDING       5776 non-null   int64 
 18  AG_DRIV        5776 non-nul

In [40]:
# count unique values in "Injury_id" column

df_KSI_clean3.groupby(['INJURY_ID']).count()

,ACCNUM,ObjectId,INVTYPE,INVAGE,VEHTYPE,MANOEUVER,DRIVACT,DRIVCOND,PEDESTRIAN,CYCLIST,AUTOMOBILE,MOTORCYCLE,TRUCK,TRSN_CITY_VEH,EMERG_VEH,PASSENGER,SPEEDING,AG_DRIV,REDLIGHT,ALCOHOL,DISABILITY,YEAR,DATE,TIME,HOUR,STREET1,STREET2,ROAD_CLASS_ID,LOCCOORD,TRAFFCTL,VISIBILITY,LIGHT,RDSFCOND,ACCLASS,IMPACTYPE,geometry,geometry2,Longitude,Latitude
INJURY_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,3886,3886,3882,3886,3845,3878,3833,3828,3886,3886,3886,3886,3886,3886,3886,3886,3886,3886,3886,3886,3886,3886,3886,3886,3886,3886,3525,3886,3863,3881,3886,3886,3886,3886,3886,3886,3886,3886,3886
2,154,154,154,154,152,153,153,153,154,154,154,154,154,154,154,154,154,154,154,154,154,154,154,154,154,154,138,154,154,154,154,154,154,154,154,154,154,154,154
3,325,325,325,325,323,325,318,319,325,325,325,325,325,325,325,325,325,325,325,325,325,325,325,325,325,325,294,325,321,324,325,325,325,325,325,325,325,325,325
4,1144,1144,1144,1144,1142,1144,1126,1128,1144,1144,1144,1144,1144,1144,1144,1144,1144,1144,1144,1144,1144,1144,1144,1144,1144,1144,1013,1144,1137,1142,1144,1144,1144,1144,1144,1144,1144,1144,1144
5,267,267,267,267,267,267,265,265,267,267,267,267,267,267,267,267,267,267,267,267,267,267,267,267,267,267,245,267,267,267,267,267,267,267,267,267,267,267,267


In [41]:
#Load Injury table
df_Injury_DB=pd.read_csv("../DB_Extracts/injury.csv")
df_Injury_DB.head(5)

,INJURY_ID,INJURY
0,1,None
1,2,Fatal
2,3,Minor
3,4,Major
4,5,Minimal


In [42]:
#drop all rows with null values

df_KSI_clean3.dropna(axis=0, how='any',inplace=True)
df_KSI_clean3.head(5)

,ACCNUM,ObjectId,INVTYPE,INVAGE,INJURY_ID,VEHTYPE,MANOEUVER,DRIVACT,DRIVCOND,PEDESTRIAN,CYCLIST,AUTOMOBILE,MOTORCYCLE,TRUCK,TRSN_CITY_VEH,EMERG_VEH,PASSENGER,SPEEDING,AG_DRIV,REDLIGHT,ALCOHOL,DISABILITY,YEAR,DATE,TIME,HOUR,STREET1,STREET2,ROAD_CLASS_ID,LOCCOORD,TRAFFCTL,VISIBILITY,LIGHT,RDSFCOND,ACCLASS,IMPACTYPE,geometry,geometry2,Longitude,Latitude
0,25301,16392,Driver,75 to 79,1,"Automobile, Station Wagon",Going Ahead,Driving Properly,Normal,1,0,1,0,0,0,0,0,0,0,0,0,0,2020,2020-01-04,1850,18,EGLINTON AVE E,BAYVIEW AVE,1,Intersection,Traffic Signal,Rain,Dark,Wet,Non-Fatal Injury,Pedestrian Collisions,"{'type': 'Point', 'coordinates': (-79.377116, ...","-79.377116, 43.710967",-79.3771,43.7109
4,37330,16506,Driver,50 to 54,1,"Automobile, Station Wagon",Going Ahead,Failed to Yield Right of Way,Normal,1,0,1,0,0,0,0,0,0,1,0,0,0,2020,2020-01-06,1555,15,DUNDAS ST W,SHERIDAN AVE,3,Intersection,Traffic Signal,Clear,"Dawn, artificial",Dry,Non-Fatal Injury,Pedestrian Collisions,"{'type': 'Point', 'coordinates': (-79.43365, 4...","-79.43365, 43.649699",-79.4336,43.6496
6,45664,16513,Driver,50 to 54,5,"Automobile, Station Wagon",Going Ahead,Driving Properly,Normal,0,0,1,0,0,0,0,1,0,1,0,0,0,2020,2020-01-07,1850,18,EGLINTON AVE E,PHARMACY AVE,1,Intersection,Traffic Signal,Clear,Dark,Dry,Non-Fatal Injury,Turning Movement,"{'type': 'Point', 'coordinates': (-79.297481, ...","-79.297481, 43.725577",-79.2974,43.7255
9,56815,16515,Driver,45 to 49,1,Delivery Van,Going Ahead,Disobeyed Traffic Control,Inattentive,0,0,1,0,0,0,0,0,1,1,1,0,0,2020,2020-01-09,1100,11,MARTIN GROVE RD,BELFIELD RD,1,Intersection,Traffic Signal,Clear,Daylight,Dry,Non-Fatal Injury,Angle,"{'type': 'Point', 'coordinates': (-79.575588, ...","-79.575588, 43.701876",-79.5755,43.7018
13,66893,16519,Driver,55 to 59,1,"Automobile, Station Wagon",Turning Left,Failed to Yield Right of Way,Inattentive,1,0,1,0,0,0,0,0,0,1,0,0,0,2020,2020-01-10,1721,17,CRUIKSHANK AVE,CARLAW AVE,2,Intersection,Stop Sign,Rain,"Dark, artificial",Wet,Non-Fatal Injury,SMV Other,"{'type': 'Point', 'coordinates': (-79.348178, ...","-79.348178, 43.680693",-79.3481,43.6806


## This section adds Redlight data to KSI dataframe

In [43]:
print(df_KSI_clean3[['ACCNUM','Longitude','Latitude']])

           ACCNUM Longitude Latitude
0           25301  -79.3771  43.7109
4           37330  -79.4336  43.6496
6           45664  -79.2974  43.7255
9           56815  -79.5755  43.7018
13          66893  -79.3481  43.6806
...           ...       ...      ...
16836  9002481518  -79.3470  43.7750
16838  9002484268  -79.3698  43.6455
16851  9002500260  -79.3848  43.6584
16853  9002501134  -79.5707  43.6915
16856  9085345312  -79.3519  43.6774

[5106 rows x 3 columns]


In [44]:
print(df_RL[['Longitude','Latitude']])

    Longitude Latitude
0    -79.3640  43.6545
1    -79.3808  43.6414
2    -79.4475  43.7920
3    -79.4292  43.7960
4    -79.6000  43.7429
..        ...      ...
209  -79.5091  43.7229
210  -79.2083  43.7879
211  -79.3502  43.6932
212  -79.1687  43.8042
213  -79.3109  43.7854

[214 rows x 2 columns]


In [45]:
test_df=df_KSI_clean3.copy()

In [46]:
test_df['IsRedlight']=0

In [47]:
def toggleredlight (AccidentID=None):
    if AccidentID:
        test_df.loc[test_df['ACCNUM']==AccidentID,"IsRedlight"]=1

In [48]:
geomerge_df=pd.merge(df_RL, df_KSI_clean3,how='inner',on=['Longitude','Latitude'])

In [49]:
geomerge['ACCNUM'].map(toggleredlight)

0      None
1      None
2      None
3      None
4      None
       ... 
262    None
263    None
264    None
265    None
266    None
Name: ACCNUM, Length: 267, dtype: object

In [50]:
geomerge['ACCNUM'].nunique()

259

In [51]:
geomerge.shape

(267, 48)

In [52]:
#print(test_df['IsRedlight'])
## count unique values in "INVTYPE" column

test_df.groupby(['IsRedlight']).count()

#pd.set_option("display.max_rows", None)
#test_df

,ACCNUM,ObjectId,INVTYPE,INVAGE,INJURY_ID,VEHTYPE,MANOEUVER,DRIVACT,DRIVCOND,PEDESTRIAN,CYCLIST,AUTOMOBILE,MOTORCYCLE,TRUCK,TRSN_CITY_VEH,EMERG_VEH,PASSENGER,SPEEDING,AG_DRIV,REDLIGHT,ALCOHOL,DISABILITY,YEAR,DATE,TIME,HOUR,STREET1,STREET2,ROAD_CLASS_ID,LOCCOORD,TRAFFCTL,VISIBILITY,LIGHT,RDSFCOND,ACCLASS,IMPACTYPE,geometry,geometry2,Longitude,Latitude
IsRedlight,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,4847,4847,4847,4847,4847,4847,4847,4847,4847,4847,4847,4847,4847,4847,4847,4847,4847,4847,4847,4847,4847,4847,4847,4847,4847,4847,4847,4847,4847,4847,4847,4847,4847,4847,4847,4847,4847,4847,4847,4847
1,259,259,259,259,259,259,259,259,259,259,259,259,259,259,259,259,259,259,259,259,259,259,259,259,259,259,259,259,259,259,259,259,259,259,259,259,259,259,259,259


In [53]:
df_KSI_clean3=test_df

## ML Random Forest

In [54]:
#drop columns which can't be used in ML model

df_KSI_clean4=df_KSI_clean3.drop(['ACCNUM','ObjectId','INVTYPE','INVAGE','YEAR','DATE','TIME','STREET1','STREET2','geometry','geometry2','Longitude','Latitude'],axis=1)
df_KSI_clean4

,INJURY_ID,VEHTYPE,MANOEUVER,DRIVACT,DRIVCOND,PEDESTRIAN,CYCLIST,AUTOMOBILE,MOTORCYCLE,TRUCK,TRSN_CITY_VEH,EMERG_VEH,PASSENGER,SPEEDING,AG_DRIV,REDLIGHT,ALCOHOL,DISABILITY,HOUR,ROAD_CLASS_ID,LOCCOORD,TRAFFCTL,VISIBILITY,LIGHT,RDSFCOND,ACCLASS,IMPACTYPE,IsRedlight
0,1,"Automobile, Station Wagon",Going Ahead,Driving Properly,Normal,1,0,1,0,0,0,0,0,0,0,0,0,0,18,1,Intersection,Traffic Signal,Rain,Dark,Wet,Non-Fatal Injury,Pedestrian Collisions,0
4,1,"Automobile, Station Wagon",Going Ahead,Failed to Yield Right of Way,Normal,1,0,1,0,0,0,0,0,0,1,0,0,0,15,3,Intersection,Traffic Signal,Clear,"Dawn, artificial",Dry,Non-Fatal Injury,Pedestrian Collisions,0
6,5,"Automobile, Station Wagon",Going Ahead,Driving Properly,Normal,0,0,1,0,0,0,0,1,0,1,0,0,0,18,1,Intersection,Traffic Signal,Clear,Dark,Dry,Non-Fatal Injury,Turning Movement,0
9,1,Delivery Van,Going Ahead,Disobeyed Traffic Control,Inattentive,0,0,1,0,0,0,0,0,1,1,1,0,0,11,1,Intersection,Traffic Signal,Clear,Daylight,Dry,Non-Fatal Injury,Angle,0
13,1,"Automobile, Station Wagon",Turning Left,Failed to Yield Right of Way,Inattentive,1,0,1,0,0,0,0,0,0,1,0,0,0,17,2,Intersection,Stop Sign,Rain,"Dark, artificial",Wet,Non-Fatal Injury,SMV Other,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16836,1,"Automobile, Station Wagon",Turning Left,Failed to Yield Right of Way,Inattentive,1,0,1,0,0,0,0,0,0,1,0,0,0,8,1,Intersection,Traffic Signal,Clear,Daylight,Wet,Non-Fatal Injury,Pedestrian Collisions,0
16838,5,"Automobile, Station Wagon",Turning Left,Improper Turn,Normal,0,0,1,0,0,0,0,1,0,0,0,0,0,21,1,Intersection,Traffic Signal,Clear,"Dark, artificial",Dry,Non-Fatal Injury,Turning Movement,0
16851,1,"Automobile, Station Wagon",Going Ahead,Driving Properly,Normal,1,0,1,0,0,0,0,0,0,1,1,0,0,15,1,Intersection,Traffic Signal,Clear,Daylight,Dry,Non-Fatal Injury,Pedestrian Collisions,0
16853,1,"Automobile, Station Wagon",Turning Left,Driving Properly,Normal,1,0,1,0,0,0,0,1,0,0,0,0,0,17,1,Intersection,Traffic Signal,Clear,Dark,Dry,Non-Fatal Injury,Pedestrian Collisions,0


In [56]:
#create a mapping dictionary, you can just enumerate the categories using a dictionary comprehension
{col: {n: cat for n, cat in enumerate(df_KSI_clean4[col].astype('category').cat.categories)}
     for col in df_KSI_clean4}

{'INJURY_ID': {0: 1, 1: 2, 2: 3, 3: 4, 4: 5},
 'VEHTYPE': {0: 'Automobile, Station Wagon',
  1: 'Bus (Other) (Go Bus, Gray Coach)',
  2: 'Construction Equipment',
  3: 'Delivery Van',
  4: 'Fire Vehicle',
  5: 'Intercity Bus',
  6: 'Moped',
  7: 'Motorcycle',
  8: 'Municipal Transit Bus (TTC)',
  9: 'Off Road - 2 Wheels',
  10: 'Other',
  11: 'Passenger Van',
  12: 'Pick Up Truck',
  13: 'Police Vehicle',
  14: 'School Bus',
  15: 'Street Car',
  16: 'Taxi',
  17: 'Tow Truck',
  18: 'Truck (other)',
  19: 'Truck - Car Carrier',
  20: 'Truck - Closed (Blazer, etc)',
  21: 'Truck - Dump',
  22: 'Truck - Open',
  23: 'Truck - Tank',
  24: 'Truck-Tractor'},
 'MANOEUVER': {0: 'Changing Lanes',
  1: 'Disabled',
  2: 'Going Ahead',
  3: 'Making U Turn',
  4: 'Merging',
  5: 'Other',
  6: 'Overtaking',
  7: 'Parked',
  8: 'Pulling Away from Shoulder or Curb',
  9: 'Pulling Onto Shoulder or towardCurb',
  10: 'Reversing',
  11: 'Slowing or Stopping',
  12: 'Stopped',
  13: 'Turning Left',
  14:

In [57]:
#convert the columns to categoricals
df_KSI_clean5=pd.DataFrame({col: df_KSI_clean4[col].astype('category').cat.codes for col in df_KSI_clean4}, index=df_KSI_clean4.index)


In [58]:
df_KSI_clean5

,INJURY_ID,VEHTYPE,MANOEUVER,DRIVACT,DRIVCOND,PEDESTRIAN,CYCLIST,AUTOMOBILE,MOTORCYCLE,TRUCK,TRSN_CITY_VEH,EMERG_VEH,PASSENGER,SPEEDING,AG_DRIV,REDLIGHT,ALCOHOL,DISABILITY,HOUR,ROAD_CLASS_ID,LOCCOORD,TRAFFCTL,VISIBILITY,LIGHT,RDSFCOND,ACCLASS,IMPACTYPE,IsRedlight
0,0,0,2,1,7,1,0,1,0,0,0,0,0,0,0,0,0,0,18,0,2,7,5,0,8,1,4,0
4,0,0,2,3,7,1,0,1,0,0,0,0,0,0,1,0,0,0,15,2,2,7,0,3,0,1,4,0
6,4,0,2,1,7,0,0,1,0,0,0,0,1,0,1,0,0,0,18,0,2,7,0,0,0,1,9,0
9,0,3,2,0,5,0,0,1,0,0,0,0,0,1,1,1,0,0,11,0,2,7,0,4,0,1,0,0
13,0,0,13,3,5,1,0,1,0,0,0,0,0,0,1,0,0,0,17,1,2,4,5,1,8,1,6,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16836,0,0,13,3,5,1,0,1,0,0,0,0,0,0,1,0,0,0,8,0,2,7,0,4,8,1,4,0
16838,4,0,13,7,7,0,0,1,0,0,0,0,1,0,0,0,0,0,21,0,2,7,0,1,0,1,9,0
16851,0,0,2,1,7,1,0,1,0,0,0,0,0,0,1,1,0,0,15,0,2,7,0,4,0,1,4,0
16853,0,0,13,1,7,1,0,1,0,0,0,0,1,0,0,0,0,0,17,0,2,7,0,0,0,1,4,0


## Random Forest

In [59]:
#Split the data into Training & Testing

KSI_df=df_KSI_clean5.copy()

# Create our features
X = KSI_df.drop("INJURY_ID", axis=1)

# Create our target
y=KSI_df["INJURY_ID"].values

y


array([0, 0, 4, ..., 0, 0, 3], dtype=int8)

In [60]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=1,stratify=y)
X_train.shape

(3829, 27)

In [61]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
balanced = BalancedRandomForestClassifier(n_estimators = 2000, random_state=1)
balanced.fit(X_train, y_train)

BalancedRandomForestClassifier(n_estimators=2000, random_state=1)

In [62]:
# Calculated the balanced accuracy score
y_pred = balanced.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.5955881728927556

In [63]:
# List the features sorted in descending order by feature importance
dict(sorted(zip(X.columns,balanced.feature_importances_), reverse=True,key = lambda record:record[1]))

{'ACCLASS': 0.14064132212678265,
 'HOUR': 0.12170250949127481,
 'IMPACTYPE': 0.08920724366763931,
 'DRIVACT': 0.0695304888086917,
 'DRIVCOND': 0.06390643225130642,
 'PEDESTRIAN': 0.05574938285807499,
 'MANOEUVER': 0.04874722292518783,
 'PASSENGER': 0.04698985574763228,
 'LIGHT': 0.04586158147769015,
 'TRAFFCTL': 0.03582318252404202,
 'ROAD_CLASS_ID': 0.034652446786334334,
 'VEHTYPE': 0.033964782036094265,
 'AG_DRIV': 0.024738952148388536,
 'MOTORCYCLE': 0.02332528341249569,
 'RDSFCOND': 0.022702005540235656,
 'SPEEDING': 0.021208458685283788,
 'LOCCOORD': 0.020302307701282316,
 'CYCLIST': 0.016116003732252967,
 'REDLIGHT': 0.015389517846151758,
 'VISIBILITY': 0.014808208924828413,
 'IsRedlight': 0.013519534179609952,
 'TRUCK': 0.010627386519657276,
 'ALCOHOL': 0.009968335100018164,
 'DISABILITY': 0.007199173753682513,
 'TRSN_CITY_VEH': 0.006819261592594865,
 'AUTOMOBILE': 0.006425977293868752,
 'EMERG_VEH': 7.314286889857828e-05}

In [64]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[690,  11,  52,  44,  64],
       [  0,  33,   1,   0,   0],
       [ 11,   6,  29,  10,  15],
       [ 21,   5,  51, 142,  31],
       [ 14,   6,  19,   8,  14]], dtype=int64)

In [65]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.94      0.80      0.89      0.86      0.84      0.71       861
          1       0.54      0.97      0.98      0.69      0.97      0.95        34
          2       0.19      0.41      0.90      0.26      0.61      0.35        71
          3       0.70      0.57      0.94      0.63      0.73      0.51       250
          4       0.11      0.23      0.91      0.15      0.46      0.19        61

avg / total       0.80      0.71      0.90      0.75      0.79      0.63      1277



## gradient boosted tree

In [66]:
from sklearn.ensemble import GradientBoostingClassifier

# Create a classifier object
learning_rates = [0.05, 0.1, 0.25, 0.5, 0.75, 1]
for learning_rate in learning_rates:
    classifier = GradientBoostingClassifier(n_estimators=20,
                                            learning_rate=learning_rate,
                                            max_features=5,
                                            max_depth=3,
                                            random_state=0)

    # Fit the model
    classifier.fit(X_train, y_train)
    print("Learning rate: ", learning_rate)

    # Score the model
    print("Accuracy score (training): {0:.3f}".format(
        classifier.score(
            X_train,
            y_train)))
    print("Accuracy score (validation): {0:.3f}".format(
        classifier.score(
            X_test,
            y_test)))
    print()

Learning rate:  0.05
Accuracy score (training): 0.770
Accuracy score (validation): 0.766

Learning rate:  0.1
Accuracy score (training): 0.792
Accuracy score (validation): 0.791

Learning rate:  0.25
Accuracy score (training): 0.818
Accuracy score (validation): 0.807

Learning rate:  0.5
Accuracy score (training): 0.837
Accuracy score (validation): 0.812

Learning rate:  0.75
Accuracy score (training): 0.850
Accuracy score (validation): 0.791

Learning rate:  1
Accuracy score (training): 0.811
Accuracy score (validation): 0.775



In [67]:
# Choose a learning rate and create classifier
classifier = GradientBoostingClassifier(n_estimators=20,
                                        learning_rate=0.5,
                                        max_features=5,
                                        max_depth=3,
                                        random_state=0)

# Fit the model
classifier.fit(X_train, y_train)

# Make Prediction
predictions = classifier.predict(X_test)
outcome_df=pd.DataFrame({"Prediction": predictions, "Actual": y_test})
outcome_df

,Prediction,Actual
0,0,0
1,0,1
2,0,0
3,0,0
4,0,0
...,...,...
1272,0,0
1273,3,3
1274,0,4
1275,0,0


In [68]:
# Calculating the accuracy score
acc_score = accuracy_score(y_test, predictions)
print(f"Accuracy Score : {acc_score}")

Accuracy Score : 0.8120595144870791


In [69]:
# Generate the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1","Actual 2", "Actual 3","Actual 4"],
    columns=["Predicted 0", "Predicted 1","Predicted 2", "Predicted 3","Predicted 4"]
)

# Displaying results
display(cm_df)

,Predicted 0,Predicted 1,Predicted 2,Predicted 3,Predicted 4
Actual 0,815,1,5,33,7
Actual 1,5,25,3,0,1
Actual 2,35,1,10,22,3
Actual 3,60,1,4,184,1
Actual 4,38,2,5,13,3


In [70]:
# Display the confusion matrix
confusion_matrix(y_test, predictions)

array([[815,   1,   5,  33,   7],
       [  5,  25,   3,   0,   1],
       [ 35,   1,  10,  22,   3],
       [ 60,   1,   4, 184,   1],
       [ 38,   2,   5,  13,   3]], dtype=int64)

In [71]:
# Generate classification report
print("Classification Report")
print(classification_report(y_test, predictions))

Classification Report
              precision    recall  f1-score   support

           0       0.86      0.95      0.90       861
           1       0.83      0.74      0.78        34
           2       0.37      0.14      0.20        71
           3       0.73      0.74      0.73       250
           4       0.20      0.05      0.08        61

    accuracy                           0.81      1277
   macro avg       0.60      0.52      0.54      1277
weighted avg       0.77      0.81      0.79      1277



## Unused code

In [72]:
# #from sklearn.preprocessing import LabelEncoder
# le = LabelEncoder()

# #'VEHTYPE','MANOEUVER','DRIVACT','DRIVCOND','LOCCOORD','TRAFFCTL','VISIBILITY','LIGHT','RDSFCOND','ACCLASS','IMPACTYPE']

# df_KSI_clean4 = df_KSI_clean3.copy()
# df_KSI_clean4['VEHTYPE_ENC'] = le.fit_transform(df_KSI_clean4['VEHTYPE'])
# df_KSI_clean4['MANOEUVER_ENC'] = le.fit_transform(df_KSI_clean4['MANOEUVER'])
# df_KSI_clean4['DRIVACT_ENC'] = le.fit_transform(df_KSI_clean4['DRIVACT'])
# df_KSI_clean4['DRIVCOND_ENC'] = le.fit_transform(df_KSI_clean4['DRIVCOND'])
# df_KSI_clean4['LOCCOORD_ENC'] = le.fit_transform(df_KSI_clean4['LOCCOORD'])
# df_KSI_clean4['TRAFFCTL_ENC'] = le.fit_transform(df_KSI_clean4['TRAFFCTL'])
# df_KSI_clean4['VISIBILITY_ENC'] = le.fit_transform(df_KSI_clean4['VISIBILITY'])
# df_KSI_clean4['LIGHT'] = le.fit_transform(df_KSI_clean4['LIGHT'])
# df_KSI_clean4['RDSFCOND'] = le.fit_transform(df_KSI_clean4['RDSFCOND'])
# df_KSI_clean4['ACCLASS'] = le.fit_transform(df_KSI_clean4['ACCLASS'])
# df_KSI_clean4['IMPACTYPE'] = le.fit_transform(df_KSI_clean4['IMPACTYPE'])
# df_KSI_clean4.head()

In [73]:
#Load Road class Table
df_RC_DB=pd.read_csv("../DB_Extracts/road_class.csv")
df_RC_DB.head(5)

,ROAD_CLASS_ID,ROAD_CLASS
0,1,Major Arterial
1,2,Collector
2,3,Minor Arterial
3,4,Local
4,5,Other


In [74]:
#Load Injury table
df_Injury_DB=pd.read_csv("../DB_Extracts/injury.csv")
df_Injury_DB.head(5)

,INJURY_ID,INJURY
0,1,None
1,2,Fatal
2,3,Minor
3,4,Major
4,5,Minimal
